In [ ]:
import math
from os import path

import astropy.units as u
import biff.scf as bscf
import gala.potential as gp
from gala.units import galactic
import h5py
import numpy as np
from scipy.optimize import minimize

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from custombiff import compute_Snlm

In [ ]:
x0 = 1.49
y0 = 0.58
z0 = 0.4

def dwek_density(x, y, z):
    """ scales are hard-coded from Wang & Zhao 2012 - see above """
    r1 = (((x/x0)**2 + (y/y0)**2)**2 + (z/z0)**4) ** 0.25
    return math.exp(-r1**2 / 2)

In [ ]:
if not path.exists('Snlm.npy') or True:
    S, Serr = compute_Snlm(dwek_density, nmax=6, lmax=6, M=1., r_s=1.)
    np.save('Snlm.npy', S)
    np.save('Snlm_err.npy', Serr)
    
S = np.load('Snlm.npy')
Serr = np.load('Snlm_err.npy')

In [ ]:
bar = bscf.SCFPotential(m=1, r_s=1.,
                        Snlm=S,
                        Tnlm=np.zeros_like(S),
                        units=galactic)

In [ ]:
grid = np.linspace(-4, 4, 128)

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
# _ = bar.plot_density_contours(grid=(grid, grid, 0.), ax=ax, )
_ = bar.plot_contours(grid=(grid, grid, 0.), ax=ax)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5), 
                         sharex=True, sharey=True)

for i in range(3):
    xyz = np.zeros((3, 1024)) + 1e-8
    xyz[i] = np.linspace(1, 100, xyz.shape[1])
    
    dens = bar.density(xyz)
    true_dens = np.array([dwek_density(x, y, z) for x,y,z in xyz.T])

    ax = axes[i]
    ax.plot(xyz[i], true_dens)
    ax.plot(xyz[i], dens)

ax.set_xscale('log')
ax.set_yscale('log')

ax.set_ylim(1e-6, 4)

---

In [ ]:
pot = gp.CCompositePotential()
pot['disk'] = gp.MiyamotoNagaiPotential(m=6E10*u.Msun,
                                               a=3*u.kpc,
                                               b=280*u.pc,
                                               units=galactic)

pot['disk2'] = gp.MiyamotoNagaiPotential(m=2E10*u.Msun,
                                                a=6.*u.kpc,
                                                b=130*u.pc,
                                                units=galactic)

pot['antidisk'] = gp.MiyamotoNagaiPotential(m=-2E10, 
                                                   a=2., b=0.28, 
                                                   units=galactic)

# pot['spher'] = gp.HernquistPotential(m=6E9*u.Msun,
#                                         c=0.25*u.kpc,
#                                         units=galactic)

pot['halo'] = gp.NFWPotential(m=6.67E11, r_s=20*u.kpc,
                                     units=galactic)

pot['bar'] = bar

In [ ]:
xyz = np.zeros((3, 1024))
xyz[0] = np.linspace(1, 25, xyz.shape[1])
vc = pot.circular_velocity(xyz)

plt.plot(xyz[0], vc.to(u.km/u.s))

In [ ]:
!rm coeffs.hdf5

In [ ]:
bar = get_bar_potential(Omega=40*u.km/u.s/u.kpc, 
                        bar_mass=8E9*u.Msun,
                        nmax=1, lmax=2, 
                        potential_no_bar=pot_no_bar)

In [ ]:
pot['bar'] = fid_bar

def func(R):
    vc = pot.circular_velocity([R,0,0.]).to(u.km/u.s).value
    return (vc - Omega * R)**2

res = minimize(func, x0=5.)

if not res.success:
    print('WARNING: Failed to find corotation radius! Hopefully you '
          'expected that...')
    return fid_bar

Rcorot = res.x[0]
Rmax = 0.8 * Rcorot # TODO: audit this choice of 0.8
print('Found corotation radius: {:.1f}'.format(Rcorot))


In [ ]:
def derp(nmax, lmax, 
         skip_odd=False, skip_even=False, skip_m=False,
         S_only=False):
    lmin = 0
    lstride = 1

    if skip_odd or skip_even:
        lstride = 2

    if skip_even:
        lmin = 1
    
    N = 0
    for n in range(nmax+1):
        for l in range(lmin, lmax+1, lstride):
            for m in range(l+1):
                if skip_m and m > 0: continue
                
                N += 1
                

    print(N)

In [ ]:
derp(5, 11, skip_odd=True)